## Step 1: Install Required Packages

In [ ]:
!pip install qiskit qiskit-machine-learning qiskit-aer scikit-learn pandas numpy

## Step 2: Upload Your Data

**Option A**: Upload the CSV files from `data/extracted_data/extracted_data/` folder

**Option B**: If you have the preprocessed data, upload `X_scaled.npy` and `y.npy`

In [ ]:
from google.colab import files
import os

# Upload CSV files (or skip if using preprocessed data)
print("Upload your climate CSV files...")
uploaded = files.upload()

# Create directory structure
os.makedirs('data/extracted_data/extracted_data', exist_ok=True)

# Move uploaded files
for filename in uploaded.keys():
    if filename.endswith('.csv'):
        !mv {filename} data/extracted_data/extracted_data/

## Step 3: Define Data Loading Function

In [ ]:
import numpy as np
import pandas as pd
import glob

def load_and_prepare_climate_data():
    """
    Load and prepare climate data from CSV files
    """
    print("=" * 70)
    print("LOADING CLIMATE DATASET")
    print("=" * 70)
    
    # Find all CSV files
    csv_files = glob.glob('data/extracted_data/extracted_data/*.csv')
    print(f"\n[INFO] Found {len(csv_files)} CSV files")
    
    if len(csv_files) == 0:
        raise FileNotFoundError("No CSV files found. Please upload your data.")
    
    # Load all CSV files
    dfs = []
    for file in csv_files:
        df = pd.read_csv(file)
        dfs.append(df)
    
    # Combine all dataframes
    combined_df = pd.concat(dfs, ignore_index=True)
    print(f"\n✓ Combined dataset shape: {combined_df.shape}")
    print(f"✓ Total samples: {len(combined_df)}")
    
    # Extract features (exclude metadata columns)
    metadata_cols = ['filename', 'latitude', 'longitude', 'altitude', 'time_index', 'time']
    feature_cols = [col for col in combined_df.columns if col not in metadata_cols]
    
    X = combined_df[feature_cols].copy()
    print(f"\n[INFO] Features extracted: {X.shape}")
    
    # Create labels based on DBZ_max > 30 dBZ (extreme weather threshold)
    if 'DBZ_max' in combined_df.columns:
        y = (combined_df['DBZ_max'] > 30).astype(int).values
        print(f"\n[INFO] Labels created based on DBZ_max > 30 dBZ")
    else:
        raise ValueError("DBZ_max column not found in dataset")
    
    # Handle missing values
    missing_count = X.isnull().sum().sum()
    if missing_count > 0:
        print(f"\n⚠️  Found {missing_count} missing values, filling with column means...")
        X = X.fillna(X.mean())
        print("✓ Missing values handled")
    
    # Display class distribution
    unique, counts = np.unique(y, return_counts=True)
    print(f"\n[INFO] Class distribution:")
    print(f"  Normal weather (0): {counts[0]} samples ({counts[0]/len(y)*100:.1f}%)")
    print(f"  Extreme weather (1): {counts[1]} samples ({counts[1]/len(y)*100:.1f}%)")
    
    print(f"\n✓ Final dataset ready:")
    print(f"  X shape: {X.shape}")
    print(f"  y shape: {y.shape}")
    
    return X.values, y

## Step 4: Quantum Ensemble Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap, PauliFeatureMap
import time
import json

print("=" * 70)
print("QUANTUM ENSEMBLE MODEL - COLAB")
print("=" * 70)

# Load data
print("\n[1/7] Loading climate data...")
X, y = load_and_prepare_climate_data()
print(f"✓ Loaded {len(X)} samples with {X.shape[1]} features")

# Normalize features
print("\n[2/7] Normalizing features...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Prepare training data (1000 samples for good balance of speed and accuracy)
print("\n[3/7] Preparing training data...")
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Sample data with natural distribution
class_0_indices = np.where(y_train == 0)[0]
class_1_indices = np.where(y_train == 1)[0]

# 1000 training samples (85% class 0, 15% class 1)
n_train_samples = 1000
n_class_0 = int(n_train_samples * 0.85)
n_class_1 = n_train_samples - n_class_0

selected_0 = np.random.choice(class_0_indices, size=n_class_0, replace=False)
selected_1 = np.random.choice(class_1_indices, size=n_class_1, replace=False)
train_indices = np.concatenate([selected_0, selected_1])
np.random.shuffle(train_indices)

X_train_sampled = X_train[train_indices]
y_train_sampled = y_train[train_indices]

# 500 test samples
test_class_0 = np.where(y_test == 0)[0]
test_class_1 = np.where(y_test == 1)[0]
n_test_samples = 500
n_test_0 = int(n_test_samples * 0.85)
n_test_1 = n_test_samples - n_test_0

selected_test_0 = np.random.choice(test_class_0, size=n_test_0, replace=False)
selected_test_1 = np.random.choice(test_class_1, size=n_test_1, replace=False)
test_indices = np.concatenate([selected_test_0, selected_test_1])
np.random.shuffle(test_indices)

X_test_sampled = X_test[test_indices]
y_test_sampled = y_test[test_indices]

print(f"✓ Training samples: {len(X_train_sampled)} (Class 0: {n_class_0}, Class 1: {n_class_1})")
print(f"✓ Test samples: {len(X_test_sampled)}")

# Create ensemble of 3 quantum models
print("\n[4/7] Building Quantum Ensemble (3 models)...")
print("⏱️  Estimated time: 15-25 minutes\n")

ensemble_predictions = []
individual_accuracies = []
individual_times = []

### Model 1: 4-qubit ZZ FeatureMap

In [ ]:
print("Model 1/3: 4-qubit ZZ FeatureMap...")
start_time_1 = time.time()

pca_4 = PCA(n_components=4)
X_train_pca4 = pca_4.fit_transform(X_train_sampled)
X_test_pca4 = pca_4.transform(X_test_sampled)

feature_map_4 = ZZFeatureMap(feature_dimension=4, reps=2, entanglement='linear')
quantum_kernel_4 = FidelityQuantumKernel(feature_map=feature_map_4)

print("  Computing quantum kernel matrices...")
K_train_4 = quantum_kernel_4.evaluate(X_train_pca4)
K_test_4 = quantum_kernel_4.evaluate(X_test_pca4, X_train_pca4)

qsvc_4 = SVC(kernel='precomputed', C=15.0, class_weight='balanced')
qsvc_4.fit(K_train_4, y_train_sampled)
y_pred_4 = qsvc_4.predict(K_test_4)
ensemble_predictions.append(y_pred_4)

acc_4 = accuracy_score(y_test_sampled, y_pred_4)
time_4 = time.time() - start_time_1
individual_accuracies.append(acc_4)
individual_times.append(time_4)
print(f"✓ Model 1: {acc_4:.4f} ({acc_4*100:.2f}%) in {time_4:.2f}s\n")

### Model 2: 5-qubit ZZ FeatureMap

In [ ]:
print("Model 2/3: 5-qubit ZZ FeatureMap...")
start_time_2 = time.time()

pca_5 = PCA(n_components=5)
X_train_pca5 = pca_5.fit_transform(X_train_sampled)
X_test_pca5 = pca_5.transform(X_test_sampled)

feature_map_5 = ZZFeatureMap(feature_dimension=5, reps=2, entanglement='full')
quantum_kernel_5 = FidelityQuantumKernel(feature_map=feature_map_5)

print("  Computing quantum kernel matrices...")
K_train_5 = quantum_kernel_5.evaluate(X_train_pca5)
K_test_5 = quantum_kernel_5.evaluate(X_test_pca5, X_train_pca5)

qsvc_5 = SVC(kernel='precomputed', C=12.0, class_weight='balanced')
qsvc_5.fit(K_train_5, y_train_sampled)
y_pred_5 = qsvc_5.predict(K_test_5)
ensemble_predictions.append(y_pred_5)

acc_5 = accuracy_score(y_test_sampled, y_pred_5)
time_5 = time.time() - start_time_2
individual_accuracies.append(acc_5)
individual_times.append(time_5)
print(f"✓ Model 2: {acc_5:.4f} ({acc_5*100:.2f}%) in {time_5:.2f}s\n")

### Model 3: 6-qubit Pauli FeatureMap

In [ ]:
print("Model 3/3: 6-qubit Pauli FeatureMap...")
start_time_3 = time.time()

pca_6 = PCA(n_components=6)
X_train_pca6 = pca_6.fit_transform(X_train_sampled)
X_test_pca6 = pca_6.transform(X_test_sampled)

feature_map_6 = PauliFeatureMap(feature_dimension=6, reps=2, paulis=['Z', 'ZZ'])
quantum_kernel_6 = FidelityQuantumKernel(feature_map=feature_map_6)

print("  Computing quantum kernel matrices...")
K_train_6 = quantum_kernel_6.evaluate(X_train_pca6)
K_test_6 = quantum_kernel_6.evaluate(X_test_pca6, X_train_pca6)

qsvc_6 = SVC(kernel='precomputed', C=10.0, class_weight='balanced')
qsvc_6.fit(K_train_6, y_train_sampled)
y_pred_6 = qsvc_6.predict(K_test_6)
ensemble_predictions.append(y_pred_6)

acc_6 = accuracy_score(y_test_sampled, y_pred_6)
time_6 = time.time() - start_time_3
individual_accuracies.append(acc_6)
individual_times.append(time_6)
print(f"✓ Model 3: {acc_6:.4f} ({acc_6*100:.2f}%) in {time_6:.2f}s\n")

## Step 5: Ensemble Voting and Results

In [ ]:
# Ensemble voting (majority vote)
print("[5/7] Combining predictions with majority voting...")
ensemble_predictions = np.array(ensemble_predictions)
y_pred_ensemble = np.apply_along_axis(
    lambda x: np.bincount(x.astype(int)).argmax(), 
    axis=0, 
    arr=ensemble_predictions
)

# Calculate ensemble metrics
print("\n[6/7] Calculating ensemble metrics...")
accuracy = accuracy_score(y_test_sampled, y_pred_ensemble)
precision = precision_score(y_test_sampled, y_pred_ensemble, zero_division=0)
recall = recall_score(y_test_sampled, y_pred_ensemble, zero_division=0)
f1 = f1_score(y_test_sampled, y_pred_ensemble, zero_division=0)
cm = confusion_matrix(y_test_sampled, y_pred_ensemble)

total_time = sum(individual_times)

print("\n[7/7] ENSEMBLE RESULTS:")
print("=" * 70)
print(f"Individual Model Accuracies:")
print(f"  Model 1 (4-qubit ZZ):    {acc_4:.4f} ({acc_4*100:.2f}%)")
print(f"  Model 2 (5-qubit ZZ):    {acc_5:.4f} ({acc_5*100:.2f}%)")
print(f"  Model 3 (6-qubit Pauli): {acc_6:.4f} ({acc_6*100:.2f}%)")
print(f"\n🎯 ENSEMBLE PERFORMANCE (Majority Voting):")
print(f"  Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"  Precision: {precision:.4f}")
print(f"  Recall:    {recall:.4f}")
print(f"  F1-Score:  {f1:.4f}")
print(f"\nConfusion Matrix:")
print(cm)
print(f"\nTotal Training Time: {total_time:.2f} seconds ({total_time/60:.1f} minutes)")
print("=" * 70)

# Compare with original QSVC
print(f"\n📊 Comparison:")
print(f"  Original QSVC (1000 samples): 78.6%")
print(f"  Ensemble (1000 samples):      {accuracy*100:.2f}%")
if accuracy > 0.786:
    improvement = (accuracy - 0.786) * 100
    print(f"  ✅ Ensemble improved by {improvement:.2f} percentage points!")
elif accuracy >= 0.78:
    print(f"  ✓ Similar performance - ensemble confirms quantum ML viability")
else:
    print(f"  → Original QSVC remains slightly better")

# Save results
results = {
    "model_type": "Quantum_Ensemble_Colab",
    "accuracy": float(accuracy),
    "precision": float(precision),
    "recall": float(recall),
    "f1_score": float(f1),
    "individual_accuracies": {
        "model_1_4qubit": float(acc_4),
        "model_2_5qubit": float(acc_5),
        "model_3_6qubit": float(acc_6)
    },
    "individual_times": {
        "model_1": float(time_4),
        "model_2": float(time_5),
        "model_3": float(time_6)
    },
    "total_time": float(total_time),
    "training_samples": int(len(X_train_sampled)),
    "test_samples": int(len(X_test_sampled)),
    "ensemble_method": "majority_voting",
    "num_models": 3
}

with open('quantum_ensemble_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("\n✓ Results saved to quantum_ensemble_results.json")
print("\n✅ ENSEMBLE TRAINING COMPLETE!")

# Download results
files.download('quantum_ensemble_results.json')